In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from bertopic import BERTopic
import nltk
import nltk.corpus
from gensim.models import Word2Vec
import os
from wordcloud import WordCloud
import collections
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))

In [10]:
# get words from the text corpus

from nltk.corpus import PlaintextCorpusReader
wordlists = PlaintextCorpusReader('da/processed', '.*')

da_words = []

for file in wordlists.fileids():
        da_words.append(wordlists.words(file))

In [11]:
# get the rosetta data

data_raw = pd.read_excel("rosetta_data_inputs_2023.xls")

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [12]:
# Get main columns and filter rows without headings
data = data_raw[["Overskrift", "Beskrivelse"]]
data = data[data["Overskrift"].notna()]

In [13]:
# remove danish words and prep data for use

def filter_danish(text):
       return " ".join(w.lower() for w in nltk.wordpunct_tokenize(text) 
                       if w.casefold() not in da_words and w.isalpha() and w != 'nan')

# combine heading and description
title_strings = data['Overskrift'].astype('str')
description_strings = data['Beskrivelse'].astype('str')
docs = title_strings + " \n\n" + description_strings

docs = docs.apply(filter_danish)

In [14]:
# tokenize text, keep short tokens and stopwords

def tokenize_text(text, tokenizer):
    tokens = tokenizer(text)  # Get tokens from text
    return tokens

tokens = docs.map(lambda x: tokenize_text(x, nltk.word_tokenize))

### BERT Topic Model

In [20]:
## BERT Topic Model

from sklearn.feature_extraction.text import CountVectorizer

# Remove stopwords after documents are assigned to topics
# Allows the transformer model to get the full context of the 
# data, while removing stopwords that are noise in the topics
vectorizer_model = CountVectorizer(stop_words="english")

# Define and train the model
topic_model = BERTopic(nr_topics=20, vectorizer_model=vectorizer_model)
topics, probs = topic_model.fit_transform(docs)
topic_model.get_topic_info() 

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1266,-1_use_cup_water_make,"[use, cup, water, make, using, surface, robot,...",[rubberband making a flexible cup that can exp...
1,0,531,0_plastic_cups_glands_packaging,"[plastic, cups, glands, packaging, people, rec...",[user designed cups people dont throw ther cup...
2,1,529,1_heat_pumice_oven_wind,"[heat, pumice, oven, wind, pizza, energy, wast...",[use waste heat for desalination of water wast...
3,2,458,2_window_windows_weeds_glass,"[window, windows, weeds, glass, old, plants, f...",[big window made of old smaller windows the bi...
4,3,242,3_tank_random_tires_tanks,"[tank, random, tires, tanks, flywheel, terrain...",[flywheels on tank in theory it s possible to ...
5,4,241,4_detect_tobacco_urine_smell,"[detect, tobacco, urine, smell, sensors, analy...",[in car cleaning gas chamber targeting the iss...
6,5,225,5_esg_employees_employee_sustainability,"[esg, employees, employee, sustainability, fas...",[physical event planner platform pepp a platfo...
7,6,212,6_berries_potatoes_cutting_pulp,"[berries, potatoes, cutting, pulp, potato, sta...",[take berries home instead of cutting strawber...
8,7,208,7_fish_chickens_feed_dead,"[fish, chickens, feed, dead, chicken, water, f...",[ask chatgpt if fish should be fed using image...
9,8,199,8_board_gypsum_boards_wall,"[board, gypsum, boards, wall, holes, screws, m...",[magnet loosens the gypsum board a magnet make...


In [21]:
topic_model.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'plastic | cups | glands | packaging | people', 531],
                                   [1, 'heat | pumice | oven | wind | pizza', 529],
                                   [2, 'window | windows | weeds | glass | old', 458],
                                   [3, 'tank | random | tires | tanks | flywheel', 242],
                                   [4, 'detect | tobacco | urine | smell | sensors', 241],
                                   [5, 'esg | employees | employee | sustainability | fashion', 225],
                                   [6, 'berries | potatoes | cutting | pulp | potato', 212],
                                   [7, 'fish | chickens | feed | dead | chicken', 208],
                                   [8, 'board | gypsum | boards | wall | holes', 199],
                                   [9, 'protein | powder | whey | alcohol | shot', 196],
                                   [10, 'robot | drone | explosives | mines | robots', 184],
                                   [11, 'chair | wheels | table | elderly | chairs', 149],
                                   [12, 'blinds | light | uv | blind | color', 142],
                                   [13, 'buns | bread | rolls | different | random', 119],
                                   [14, 'bed | matress | pressure | patient | mattress', 118],
                                   [15, 'lego | bricks | model | idea | good', 117],
                                   [16, 'hearing | aids | aid | sound | noise', 105],
                                   [17, 'aerodynamic | rider | bearing | efficiency | chain', 85],
                                   [18, 'stent | define | kirigami | removeal | bit', 13]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': array([531, 529, 458, 242, 241, 225, 212, 208, 199, 196, 184, 149, 142, 119,
                                        118, 117, 105,  85,  13]),
                         'sizemode': 'area',
                         'sizeref': 0.331875,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([ 7.859595 ,  9.055028 ,  9.07061  , -4.574169 ,  9.087479 ,  8.222476 ,
                           7.663743 , -4.314648 , -5.953236 ,  8.822505 , -5.2268734, -4.2596736,
                           8.722328 ,  8.478016 , -4.0707226, -5.6319885, -5.1358833, -3.8249772,
                           8.09429  ], dtype=float32),
              'xaxis': 'x',
              'y': array([  9.950859 ,  10.555099 ,  10.307845 , -10.697761 ,   8.484    ,
                            9.86368  ,   9.5412035, -10.837271 , -11.03106  ,   8.737999 ,
                          -11.132302 , -10.073792 ,  10.546716 ,   9.711806 ,  -9.915168 ,
                          -11.244517 , -11.429838 , -10.943216 ,   9.119545 ], dtype=float32),
              'yaxis': 'y'}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'D1',
                                'x': -6.846221518516541,
                                'y': -0.5029752969741823,
                                'yshift': 10},
                               {'showarrow': False,
                                'text': 'D2',
                                'x': 1.8021894574165347,
                                'xshift': 10,
                                'y': 12.138363313674926}],
               'height': 650,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'itemsizing': 'constant', 'tracegroupgap': 0},
               'margin': {'